In [1]:
import pandas as pd

In [2]:
base_path = './합본'
years = ['/22', '/23']
regions = ['/수도권', '/동부', '/서부', '/제주산간']
csv_types = ['/TL_csv/', '/VL_csv/']

file_names = ['tn_visit_area_info_방문지정보', 'tn_traveller_master_여행객 Master', 'tn_travel_여행', 'tn_tour_photo_관광사진',
            'tn_mvmn_consume_his_이동수단소비내역', 'tn_move_his_이동내역', 'tn_lodge_consume_his_숙박소비내역', 'tn_companion_info_동반자정보',
             'tn_adv_consume_his_사전소비내역', 'tn_activity_his_활동내역', 'tn_activity_consume_his_활동소비내역' ]

year_region_codes = {
    '/22': ['_A', '_B', '_C', '_D'],  # 22년도 수도권 A, 동부 B, 서부 C, 제주산간 D
    '/23': ['_E', '_F', '_G', '_H']   # 23년도 수도권 E, 동부 F, 서부 G, 제주산간 H
}

m_df =[]
year = '/23'

for file_name in file_names:
    dfs = []
    for region, code in zip(regions, year_region_codes[year]):
        for csv_type in csv_types:
            file_path = f"{base_path}{year}{region}{csv_type}{file_name}{code}.csv"
            df = pd.read_csv(file_path)
            dfs.append(df)
    mdf = pd.concat(dfs)
    m_df.append(mdf)
m_df_dict = {file_name: df for file_name, df in zip(file_names, m_df)} 

In [3]:
lodgings = m_df_dict['tn_lodge_consume_his_숙박소비내역']

In [11]:
lodgings_t = m_df_dict['tn_visit_area_info_방문지정보']
lodgings_t = lodgings_t[lodgings_t['VISIT_AREA_TYPE_CD'] == 24]

In [41]:
lodgings_t.to_csv("lodgings.csv", index=False)

In [10]:
lodgings.count()

TRAVEL_ID              7482
LODGING_NM             7482
LODGING_PAYMENT_SEQ    7482
LODGING_TYPE_CD        7482
RSVT_YN                7482
CHK_IN_DT_MIN             0
CHK_OUT_DT_MIN            0
PAYMENT_NUM            7482
BRNO                      0
STORE_NM               4267
ROAD_NM_ADDR           6558
LOTNO_ADDR             5198
ROAD_NM_CD             3505
LOTNO_CD               3505
PAYMENT_DT             4228
PAYMENT_MTHD_SE        7482
PAYMENT_AMT_WON        7481
PAYMENT_ETC            3789
dtype: int64

In [11]:
lodgings_t.count()

VISIT_AREA_ID          13183
TRAVEL_ID              13183
VISIT_ORDER            13183
VISIT_AREA_NM          13183
VISIT_START_YMD        13183
VISIT_END_YMD          13183
ROAD_NM_ADDR           12604
LOTNO_ADDR             11850
X_COORD                11726
Y_COORD                11726
ROAD_NM_CD                27
LOTNO_CD                  27
POI_ID                 11763
POI_NM                 11763
RESIDENCE_TIME_MIN     13122
VISIT_AREA_TYPE_CD     13183
REVISIT_YN             13122
VISIT_CHC_REASON_CD    13122
LODGING_TYPE_CD        13119
DGSTFN                 13122
REVISIT_INTENTION      13122
RCMDTN_INTENTION       13122
SGG_CD                  1433
dtype: int64

숙박소비내역 및 방문지 정보는 서로 완전히 같지 않았다 
 - 숙박 소비 내역 및 방문지 정보에 있는 것들 -> 방문지 정보의 수량이 더 많은 것으로 보아 한 숙박업소에서 여러 날을 묵었을 때 이것이 묶여서 처리된 것이 숙박소비내역으로 보임
 - 숙박 소비 내역에만 있는 것들 -> 방문지 정보에 있긴 할 테지만 24번 카테고리로 분류되지 않았을 것으로 생각됨 
 - 방문지 정보에만 있는 것들 -> 어째서 숙박 소비 내역으로 처리되지 않았는지 확인 
이 3가지를 Travel_id를 기준으로 해서 하나의 데이터로 통합해볼 필요가 있음 

In [12]:
lodgings_t['VISIT_START_YMD'] = pd.to_datetime(lodgings_t['VISIT_START_YMD'])
lodgings_t['VISIT_END_YMD'] = pd.to_datetime(lodgings_t['VISIT_END_YMD'])

lodgings_t['SLEEP'] = (lodgings_t['VISIT_END_YMD'] - lodgings_t['VISIT_START_YMD']).dt.days

C:\Users\PC\AppData\Local\Temp\ipykernel_45172\1308013072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lodgings_t['VISIT_START_YMD'] = pd.to_datetime(lodgings_t['VISIT_START_YMD'])
C:\Users\PC\AppData\Local\Temp\ipykernel_45172\1308013072.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lodgings_t['VISIT_END_YMD'] = pd.to_datetime(lodgings_t['VISIT_END_YMD'])
C:\Users\PC\AppData\Local\Temp\ipykernel_45172\1308013072.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [36]:
lodgings_t

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,SLEEP,TRAVELER_ID,MVMN_NM,TRAVEL_STATUS_ACCOMPANY
0,2304290007,e_e000009,6,호텔 프라하,2023-04-29,2023-04-30,경기 평택시 통복로26번길 11,경기 평택시 통복동 114-10,127.083338,36.994097,...,8.0,2.0,1.0,1.0,1.0,NaN,1,e000009,자가용,2인 여행(가족 외)
1,2304280005,e_e000013,5,해브펀 캠핑 어라운드,2023-04-28,2023-04-29,경기 파주시 월롱면 도감로 130,경기 파주시 월롱면 위전리 89-1,126.799751,37.790896,...,11.0,5.0,5.0,5.0,5.0,NaN,1,e000013,자가용,부모 동반 여행
2,2304290005,e_e000020,5,파르티아 워터하우스,2023-04-29,2023-04-29,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,8.0,1.0,4.0,3.0,4.0,NaN,0,e000020,자가용,나홀로 여행
3,2304290007,e_e000020,7,파르티아 워터하우스,2023-04-29,2023-04-30,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,8.0,1.0,4.0,3.0,4.0,NaN,1,e000020,자가용,나홀로 여행
4,2304290004,e_e000021,6,호텔 헤르메스,2023-04-29,2023-04-29,경기 화성시 남양읍 역골로 9-28,경기 화성시 남양읍 남양리 2076,126.827220,37.200436,...,7.0,1.0,4.0,4.0,4.0,NaN,0,e000021,자가용,2인 여행(가족 외)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13178,2309170006,h_h006367,5,유지 공간,2023-09-17,2023-09-18,제주특별자치도 제주시 한림읍 금능남1길 22,제주특별자치도 제주시 한림읍 금능리 2088-8,126.231535,33.385660,...,6.0,7.0,5.0,5.0,4.0,NaN,1,h006367,자가용,나홀로 여행
13179,2309190001,h_h006595,1,숙소,2023-09-19,2023-09-19,제주특별자치도 제주시 수덕1길 7,제주특별자치도 제주시 노형동 1288-4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,h006595,자가용,2인 여행(가족 외)
13180,2309190002,h_h006595,6,온예가,2023-09-19,2023-09-20,제주특별자치도 서귀포시 성산읍 온평애향로 68-6,제주특별자치도 서귀포시 성산읍 온평리 961-1,126.903461,33.405199,...,2.0,8.0,5.0,4.0,4.0,NaN,1,h006595,자가용,2인 여행(가족 외)
13181,2309160002,h_h006639,6,제이 뷰 호텔,2023-09-16,2023-09-17,제주특별자치도 서귀포시 중산간서로 935,제주특별자치도 서귀포시 색달동 1622-2,126.416451,33.264705,...,8.0,1.0,4.0,4.0,4.0,NaN,1,h006639,자가용,2인 여행(가족 외)


In [ ]:
lodgings_t = pd.merge(lodgings_t, m_df_dict['tn_travel_여행'][['TRAVEL_ID', 'TRAVELER_ID', 'MVMN_NM']], on='TRAVEL_ID', how='left')
lodgings_t = pd.merge(lodgings_t, m_df_dict['tn_traveller_master_여행객 Master'][['TRAVELER_ID','TRAVEL_STATUS_ACCOMPANY']], on='TRAVELER_ID', how='left')
lodgings_t['MVMN_NM'].fillna("자가용", inplace=True)


In [35]:
lodgings_t['MVMN_NM'].value_counts()

MVMN_NM
자가용       6627
대중교통 등    6556
Name: count, dtype: int64

In [ ]:
grouped_lodgings = lodgings_t.groupby(['TRAVEL_ID', 'VISIT_AREA_NM']).agg({
    'VISIT_AREA_NM' : 'count',
    'DGSTFN': 'mean',
    'REVISIT_INTENTION' : 'mean',
    'RCMDTN_INTENTION': 'mean',
    'LODGING_TYPE_CD' : 'first',
    'X_COORD' : 'first',
    'Y_COORD' : 'first',
    'SLEEP' : 'sum'
})
grouped_lodgings.rename(columns={'VISIT_AREA_NM' : 'COUNT'}, inplace=True)

In [73]:
grouped_lodgings

COUNT  DGSTFN  REVISIT_INTENTION  RCMDTN_INTENTION  \
TRAVEL_ID VISIT_AREA_NM                                                        
e_e000005 갤럭시호텔               2     5.0                5.0               5.0   
          더 위크앤 리조트           2     5.0                5.0               5.0   
          호텔 스카이파크 인천 송도      2     5.0                5.0               5.0   
e_e000009 호텔 프라하              1     1.0                1.0               1.0   
e_e000013 해브펀 캠핑 어라운드         1     5.0                5.0               5.0   
...                         ...     ...                ...               ...   
h_h006639 제이 뷰 호텔             1     4.0                4.0               4.0   
h_h006653 숙소                  1     NaN                NaN               NaN   
h_h006665 숙소                  1     NaN                NaN               NaN   
h_h006748 롯데호텔 제주             2     5.0                5.0               5.0   
h_h006753 롯데호텔 제주             1     5.0                5.0               5.0   

                          LODGING_TYPE_CD     X_COORD    Y_COORD  SLEEP  
TRAVEL_ID VISIT_AREA_NM                                                  
e_e000005 갤럭시호텔                       2.0  126.599158  37.476967      1  
          더 위크앤 리조트                   4.0  126.370757  37.450624      1  
          호텔 스카이파크 인천 송도              1.0  126.638714  37.384631      1  
e_e000009 호텔 프라하                      2.0  127.083338  36.994097      1  
e_e000013 해브펀 캠핑 어라운드                 5.0  126.799751  37.790896      1  
...                                   ...         ...        ...    ...  
h_h006639 제이 뷰 호텔                     1.0  126.416451  33.264705      1  
h_h006653 숙소                          NaN         NaN        NaN      0  
h_h006665 숙소                          NaN         NaN        NaN      0  
h_h006748 롯데호텔 제주                     1.0  126.410600  33.248457      1  
h_h006753 롯데호텔 제주                     1.0  126.410600  33.248457      1  

[7427 rows x 8 columns]

In [84]:
grouped_lodgings.reset_index(inplace=True)

In [86]:
grouped_lodgings.set_index(['TRAVEL_ID'])

,VISIT_AREA_NM,COUNT,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,LODGING_TYPE_CD,X_COORD,Y_COORD,SLEEP
TRAVEL_ID,,,,,,,,,
e_e000005,갤럭시호텔,2,5.0,5.0,5.0,2.0,126.599158,37.476967,1
e_e000005,더 위크앤 리조트,2,5.0,5.0,5.0,4.0,126.370757,37.450624,1
e_e000005,호텔 스카이파크 인천 송도,2,5.0,5.0,5.0,1.0,126.638714,37.384631,1
e_e000009,호텔 프라하,1,1.0,1.0,1.0,2.0,127.083338,36.994097,1
e_e000013,해브펀 캠핑 어라운드,1,5.0,5.0,5.0,5.0,126.799751,37.790896,1
...,...,...,...,...,...,...,...,...,...
h_h006639,제이 뷰 호텔,1,4.0,4.0,4.0,1.0,126.416451,33.264705,1
h_h006653,숙소,1,NaN,NaN,NaN,NaN,NaN,NaN,0
h_h006665,숙소,1,NaN,NaN,NaN,NaN,NaN,NaN,0


In [98]:
grouped_lodgings[~grouped_lodgings['X_COORD'].isna()][grouped_lodgings[~grouped_lodgings['X_COORD'].isna()]['VISIT_AREA_NM'].str.contains('숙소', na=False)]

,TRAVEL_ID,VISIT_AREA_NM,COUNT,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,LODGING_TYPE_CD,X_COORD,Y_COORD,SLEEP
2725,f_f008629,에어비앤비 숙소,1,5.0,5.0,5.0,7.0,129.209645,35.839335,1
4274,g_g004929,에어비엔비 전주 숙소,1,1.0,5.0,5.0,11.0,127.153452,35.819845,0
4275,g_g004929,전주 숙소,1,1.0,4.0,5.0,11.0,127.153452,35.819845,1
5782,h_h001415,호아 요가 숙소,5,5.0,5.0,5.0,3.0,126.313432,33.460147,2


In [99]:
grouped_lodgings = grouped_lodgings[~grouped_lodgings['X_COORD'].isna()]

In [100]:
grouped_lodgings

,TRAVEL_ID,VISIT_AREA_NM,COUNT,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,LODGING_TYPE_CD,X_COORD,Y_COORD,SLEEP
0,e_e000005,갤럭시호텔,2,5.0,5.0,5.0,2.0,126.599158,37.476967,1
1,e_e000005,더 위크앤 리조트,2,5.0,5.0,5.0,4.0,126.370757,37.450624,1
2,e_e000005,호텔 스카이파크 인천 송도,2,5.0,5.0,5.0,1.0,126.638714,37.384631,1
3,e_e000009,호텔 프라하,1,1.0,1.0,1.0,2.0,127.083338,36.994097,1
4,e_e000013,해브펀 캠핑 어라운드,1,5.0,5.0,5.0,5.0,126.799751,37.790896,1
...,...,...,...,...,...,...,...,...,...,...
7418,h_h006587,가문이 메,2,4.5,4.0,4.5,3.0,126.491620,33.254587,1
7420,h_h006595,온예가,1,5.0,4.0,4.0,8.0,126.903461,33.405199,1
7422,h_h006639,제이 뷰 호텔,1,4.0,4.0,4.0,1.0,126.416451,33.264705,1
7425,h_h006748,롯데호텔 제주,2,5.0,5.0,5.0,1.0,126.410600,33.248457,1


In [44]:
cnt_l = lodgings.groupby(['TRAVEL_ID', 'LODGING_NM']).agg({
    'LODGING_NM' : 'count',
})
cnt_l

LODGING_NM
TRAVEL_ID LODGING_NM              
e_e000005 갤럭시호텔                  1
          더위크앤 리조트               1
          호텔스카이파크 인천송도           1
e_e000013 해브펀캠핑어라운드              1
e_e000020 파로티아워터하우스              1
...                            ...
h_h006639 제이뷰호텔                  1
h_h006665 유유한가 펜션                1
          집                      1
h_h006748 롯데호텔 제주                1
h_h006753 롯데호텔 제주                1

[7167 rows x 1 columns]

In [30]:
merged_cnt = pd.merge(cnt_l, cnt_t, left_index=True, right_index=True, how='outer')

In [31]:
merged_cnt

,TRAVEL_ID_x,TRAVEL_ID_y
TRAVEL_ID,,
e_e000005,3.0,6.0
e_e000009,NaN,1.0
e_e000013,1.0,1.0
e_e000020,1.0,2.0
e_e000021,1.0,2.0
...,...,...
h_h006639,1.0,1.0
h_h006653,NaN,1.0
h_h006665,2.0,1.0


In [51]:
grouped_lodgings = grouped_lodgings[~grouped_lodgings['X_COORD'].isna()]

In [52]:
grouped_lodgings

VISIT_AREA_NM  DGSTFN  REVISIT_INTENTION  \
TRAVEL_ID VISIT_AREA_NM                                              
e_e000005 갤럭시호텔                       2     5.0                5.0   
          더 위크앤 리조트                   2     5.0                5.0   
          호텔 스카이파크 인천 송도              2     5.0                5.0   
e_e000009 호텔 프라하                      1     1.0                1.0   
e_e000013 해브펀 캠핑 어라운드                 1     5.0                5.0   
...                                 ...     ...                ...   
h_h006587 가문이 메                       2     4.5                4.0   
h_h006595 온예가                         1     5.0                4.0   
h_h006639 제이 뷰 호텔                     1     4.0                4.0   
h_h006748 롯데호텔 제주                     2     5.0                5.0   
h_h006753 롯데호텔 제주                     1     5.0                5.0   

                          RCMDTN_INTENTION     X_COORD    Y_COORD  
TRAVEL_ID VISIT_AREA_NM                                            
e_e000005 갤럭시호텔                        5.0  126.599158  37.476967  
          더 위크앤 리조트                    5.0  126.370757  37.450624  
          호텔 스카이파크 인천 송도               5.0  126.638714  37.384631  
e_e000009 호텔 프라하                       1.0  127.083338  36.994097  
e_e000013 해브펀 캠핑 어라운드                  5.0  126.799751  37.790896  
...                                    ...         ...        ...  
h_h006587 가문이 메                        4.5  126.491620  33.254587  
h_h006595 온예가                          4.0  126.903461  33.405199  
h_h006639 제이 뷰 호텔                      4.0  126.416451  33.264705  
h_h006748 롯데호텔 제주                      5.0  126.410600  33.248457  
h_h006753 롯데호텔 제주                      5.0  126.410600  33.248457  

[6525 rows x 6 columns]

In [ ]:
lodging_data = lodgings_t


from sklearn.preprocessing import MinMaxScaler

# DGSTFN 정규화
scaler = MinMaxScaler()
lodging_data['NORMALIZED_DGSTFN'] = scaler.fit_transform(lodging_data[['DGSTFN']])

# 보정 함수 적용
# REVISIT_INTENTION based on RCMDTN_INTENTION
lodging_data['CORRECTED_REVISIT_INTENTION'] = lodging_data['REVISIT_INTENTION'] * (
    1 - 0.1 * (lodging_data['RCMDTN_INTENTION'] < 3).astype(float)
)

# RCMDTN_INTENTION based on REVISIT_INTENTION
lodging_data['CORRECTED_RCMDTN_INTENTION'] = lodging_data['RCMDTN_INTENTION'] * (
    1 - 0.1 * (lodging_data['REVISIT_INTENTION'] < 3).astype(float)
)

# 가중치 계산
lodging_data['FINAL_RECOMMENDATION_SCORE'] = (
    0.20 * lodging_data['REVISIT_YN'].replace({'Y': 1, 'N': 0}) +
    0.35 * lodging_data['NORMALIZED_DGSTFN'] +
    0.25 * lodging_data['CORRECTED_REVISIT_INTENTION'] +
    0.20 * lodging_data['CORRECTED_RCMDTN_INTENTION']
)

    

C:\Users\PC\AppData\Local\Temp\ipykernel_41356\1763346212.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lodging_data['NORMALIZED_DGSTFN'] = scaler.fit_transform(lodging_data[['DGSTFN']])
C:\Users\PC\AppData\Local\Temp\ipykernel_41356\1763346212.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lodging_data['CORRECTED_REVISIT_INTENTION'] = lodging_data['REVISIT_INTENTION'] * (
C:\Users\PC\AppData\Local\Temp\ipykernel_41356\1763346212.py:17: SettingWithCopyWarning: 
A value is trying to be set on a c

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,SLEEP,NORMALIZED_DGSTFN,CORRECTED_REVISIT_INTENTION,CORRECTED_RCMDTN_INTENTION,FINAL_RECOMMENDATION_SCORE
21,2304290007,e_e000009,6,호텔 프라하,2023-04-29,2023-04-30,경기 평택시 통복로26번길 11,경기 평택시 통복동 114-10,127.083338,36.994097,...,2.0,1.0,1.0,1.0,NaN,1,0.00,0.9,0.9,0.4050
61,2304280005,e_e000013,5,해브펀 캠핑 어라운드,2023-04-28,2023-04-29,경기 파주시 월롱면 도감로 130,경기 파주시 월롱면 위전리 89-1,126.799751,37.790896,...,5.0,5.0,5.0,5.0,NaN,1,1.00,5.0,5.0,2.6000
104,2304290005,e_e000020,5,파르티아 워터하우스,2023-04-29,2023-04-29,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,1.0,4.0,3.0,4.0,NaN,0,0.75,3.0,4.0,1.8125
106,2304290007,e_e000020,7,파르티아 워터하우스,2023-04-29,2023-04-30,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,1.0,4.0,3.0,4.0,NaN,1,0.75,3.0,4.0,1.8125
116,2304290004,e_e000021,6,호텔 헤르메스,2023-04-29,2023-04-29,경기 화성시 남양읍 역골로 9-28,경기 화성시 남양읍 남양리 2076,126.827220,37.200436,...,1.0,4.0,4.0,4.0,NaN,0,0.75,4.0,4.0,2.0625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5453,2309170006,h_h006367,5,유지 공간,2023-09-17,2023-09-18,제주특별자치도 제주시 한림읍 금능남1길 22,제주특별자치도 제주시 한림읍 금능리 2088-8,126.231535,33.385660,...,7.0,5.0,5.0,4.0,NaN,1,1.00,5.0,4.0,2.4000
5502,2309190001,h_h006595,1,숙소,2023-09-19,2023-09-19,제주특별자치도 제주시 수덕1길 7,제주특별자치도 제주시 노형동 1288-4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
5503,2309190002,h_h006595,6,온예가,2023-09-19,2023-09-20,제주특별자치도 서귀포시 성산읍 온평애향로 68-6,제주특별자치도 서귀포시 성산읍 온평리 961-1,126.903461,33.405199,...,8.0,5.0,4.0,4.0,NaN,1,1.00,4.0,4.0,2.1500
5521,2309160002,h_h006639,6,제이 뷰 호텔,2023-09-16,2023-09-17,제주특별자치도 서귀포시 중산간서로 935,제주특별자치도 서귀포시 색달동 1622-2,126.416451,33.264705,...,1.0,4.0,4.0,4.0,NaN,1,0.75,4.0,4.0,2.0625


In [109]:
na_lodging_data = lodging_data[lodging_data['FINAL_RECOMMENDATION_SCORE'].isna()]
na_lodging_data[(na_lodging_data['VISIT_AREA_NM'].str.contains('숙소', na=False))]

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,SLEEP,NORMALIZED_DGSTFN,CORRECTED_REVISIT_INTENTION,CORRECTED_RCMDTN_INTENTION,FINAL_RECOMMENDATION_SCORE
489,2305270001,e_e000346,1,숙소,2023-05-27,2023-05-27,어답산로 516 병지방 오토캠핑장,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.273034e+09,0,NaN,NaN,NaN,NaN
4562,2305280001,e_e001499,1,숙소,2023-05-28,2023-05-28,서울 송파구 오금로11길 47,서울 송파구 방이동 46-3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
5221,2306030001,e_e002051,1,숙소,2023-06-03,2023-06-03,경기 용인시 기흥구 삼성로 1 삼성전자 기숙사,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
8115,2306170001,e_e003245,1,숙소,2023-06-17,2023-06-17,경기 부천시 조마루로297번길 13,경기 부천시 중동 1130-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
8271,2306170001,e_e003289,1,숙소,2023-06-17,2023-06-17,인천 남동구 석촌로46번길 33,인천 남동구 간석동 168-3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3483,2307300001,h_h002249,1,숙소,2023-07-30,2023-07-30,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.150011e+09,0,NaN,NaN,NaN,NaN
4550,2308290001,h_h004058,1,숙소,2023-08-29,2023-08-29,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.653010e+09,0,NaN,NaN,NaN,NaN
5326,2309160001,h_h006139,1,숙소,2023-09-16,2023-09-16,경기 성남시 분당구 돌마로 87 청운고시원,경기 성남시 분당구 금곡동 151,NaN,NaN,...,NaN,NaN,NaN,NaN,4.113511e+09,0,NaN,NaN,NaN,NaN
5378,2309160001,h_h006234,1,숙소,2023-09-16,2023-09-16,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.150011e+09,0,NaN,NaN,NaN,NaN


In [39]:
lodgings_t['TRAVEL_STATUS_ACCOMPANY'].value_counts()

TRAVEL_STATUS_ACCOMPANY
2인 여행(가족 외)           4977
자녀 동반 여행              2053
나홀로 여행                2011
3인 이상 여행(가족 외)        1820
2인 가족 여행              1311
부모 동반 여행               624
3대 동반 여행(친척 포함)        374
3인 이상 가족 여행(친척 포함)      13
Name: count, dtype: int64

In [ ]:
accompany_mapping = {
    '2인 여행(가족 외)' : 0,
    '자녀 동반 여행' : 1,
    '나홀로 여행' : 0,
    '3인 이상 여행(가족 외)' : 0,
    '2인 가족 여행' : 1,
    '부모 동반 여행' : 1,
    '3대 동반 여행(친척 포함)' : 1,
    '3인 이상 가족 여행(친척 포함)' : 1
}

In [44]:
lodgings_t['TRAVEL_STATUS_ACCOMPANY'] = lodgings_t['TRAVEL_STATUS_ACCOMPANY'].map(accompany_mapping)
lodgings_t

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,SLEEP,TRAVELER_ID,MVMN_NM,TRAVEL_STATUS_ACCOMPANY
0,2304290007,e_e000009,6,호텔 프라하,2023-04-29,2023-04-30,경기 평택시 통복로26번길 11,경기 평택시 통복동 114-10,127.083338,36.994097,...,8.0,2.0,1.0,1.0,1.0,NaN,1,e000009,자가용,NaN
1,2304280005,e_e000013,5,해브펀 캠핑 어라운드,2023-04-28,2023-04-29,경기 파주시 월롱면 도감로 130,경기 파주시 월롱면 위전리 89-1,126.799751,37.790896,...,11.0,5.0,5.0,5.0,5.0,NaN,1,e000013,자가용,NaN
2,2304290005,e_e000020,5,파르티아 워터하우스,2023-04-29,2023-04-29,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,8.0,1.0,4.0,3.0,4.0,NaN,0,e000020,자가용,NaN
3,2304290007,e_e000020,7,파르티아 워터하우스,2023-04-29,2023-04-30,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,8.0,1.0,4.0,3.0,4.0,NaN,1,e000020,자가용,NaN
4,2304290004,e_e000021,6,호텔 헤르메스,2023-04-29,2023-04-29,경기 화성시 남양읍 역골로 9-28,경기 화성시 남양읍 남양리 2076,126.827220,37.200436,...,7.0,1.0,4.0,4.0,4.0,NaN,0,e000021,자가용,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13178,2309170006,h_h006367,5,유지 공간,2023-09-17,2023-09-18,제주특별자치도 제주시 한림읍 금능남1길 22,제주특별자치도 제주시 한림읍 금능리 2088-8,126.231535,33.385660,...,6.0,7.0,5.0,5.0,4.0,NaN,1,h006367,자가용,NaN
13179,2309190001,h_h006595,1,숙소,2023-09-19,2023-09-19,제주특별자치도 제주시 수덕1길 7,제주특별자치도 제주시 노형동 1288-4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,h006595,자가용,NaN
13180,2309190002,h_h006595,6,온예가,2023-09-19,2023-09-20,제주특별자치도 서귀포시 성산읍 온평애향로 68-6,제주특별자치도 서귀포시 성산읍 온평리 961-1,126.903461,33.405199,...,2.0,8.0,5.0,4.0,4.0,NaN,1,h006595,자가용,NaN
13181,2309160002,h_h006639,6,제이 뷰 호텔,2023-09-16,2023-09-17,제주특별자치도 서귀포시 중산간서로 935,제주특별자치도 서귀포시 색달동 1622-2,126.416451,33.264705,...,8.0,1.0,4.0,4.0,4.0,NaN,1,h006639,자가용,NaN


In [47]:
mvmn_map = {
    '자가용' : 0,
    '대중교통 등' : 1
}
lodgings_t['MVMN_NM'] = lodgings_t['MVMN_NM'].map(mvmn_map)

In [48]:
lodgings_t

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,SLEEP,TRAVELER_ID,MVMN_NM,TRAVEL_STATUS_ACCOMPANY
0,2304290007,e_e000009,6,호텔 프라하,2023-04-29,2023-04-30,경기 평택시 통복로26번길 11,경기 평택시 통복동 114-10,127.083338,36.994097,...,8.0,2.0,1.0,1.0,1.0,NaN,1,e000009,0,NaN
1,2304280005,e_e000013,5,해브펀 캠핑 어라운드,2023-04-28,2023-04-29,경기 파주시 월롱면 도감로 130,경기 파주시 월롱면 위전리 89-1,126.799751,37.790896,...,11.0,5.0,5.0,5.0,5.0,NaN,1,e000013,0,NaN
2,2304290005,e_e000020,5,파르티아 워터하우스,2023-04-29,2023-04-29,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,8.0,1.0,4.0,3.0,4.0,NaN,0,e000020,0,NaN
3,2304290007,e_e000020,7,파르티아 워터하우스,2023-04-29,2023-04-30,경기 가평군 설악면 미사리로 707-96,경기 가평군 설악면 미사리 500-1,127.540699,37.717804,...,8.0,1.0,4.0,3.0,4.0,NaN,1,e000020,0,NaN
4,2304290004,e_e000021,6,호텔 헤르메스,2023-04-29,2023-04-29,경기 화성시 남양읍 역골로 9-28,경기 화성시 남양읍 남양리 2076,126.827220,37.200436,...,7.0,1.0,4.0,4.0,4.0,NaN,0,e000021,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13178,2309170006,h_h006367,5,유지 공간,2023-09-17,2023-09-18,제주특별자치도 제주시 한림읍 금능남1길 22,제주특별자치도 제주시 한림읍 금능리 2088-8,126.231535,33.385660,...,6.0,7.0,5.0,5.0,4.0,NaN,1,h006367,0,NaN
13179,2309190001,h_h006595,1,숙소,2023-09-19,2023-09-19,제주특별자치도 제주시 수덕1길 7,제주특별자치도 제주시 노형동 1288-4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,h006595,0,NaN
13180,2309190002,h_h006595,6,온예가,2023-09-19,2023-09-20,제주특별자치도 서귀포시 성산읍 온평애향로 68-6,제주특별자치도 서귀포시 성산읍 온평리 961-1,126.903461,33.405199,...,2.0,8.0,5.0,4.0,4.0,NaN,1,h006595,0,NaN
13181,2309160002,h_h006639,6,제이 뷰 호텔,2023-09-16,2023-09-17,제주특별자치도 서귀포시 중산간서로 935,제주특별자치도 서귀포시 색달동 1622-2,126.416451,33.264705,...,8.0,1.0,4.0,4.0,4.0,NaN,1,h006639,0,NaN


In [4]:
lodging_data= m_df_dict['tn_visit_area_info_방문지정보']
lodging_data= lodging_data[lodging_data['VISIT_AREA_TYPE_CD'] == 24]

# 숙박일수를 통해 같은 이름의 숙박 업소가 여러번 나오더라도 실제 숙박 일수를 알아내기 위해 추가
lodging_data['VISIT_START_YMD'] = pd.to_datetime(lodging_data['VISIT_START_YMD'])
lodging_data['VISIT_END_YMD'] = pd.to_datetime(lodging_data['VISIT_END_YMD'])
lodging_data['SLEEP'] = (lodging_data['VISIT_END_YMD'] - lodging_data['VISIT_START_YMD']).dt.days

lodging_data = pd.merge(lodging_data, m_df_dict['tn_travel_여행'][['TRAVEL_ID', 'TRAVELER_ID', 'MVMN_NM']], on='TRAVEL_ID', how='left')
lodging_data = pd.merge(lodging_data, m_df_dict['tn_traveller_master_여행객 Master'][['TRAVELER_ID','TRAVEL_STATUS_ACCOMPANY']], on='TRAVELER_ID', how='left')
lodging_data['MVMN_NM'].fillna("자가용", inplace=True)

# 가족 여행 횟수만 세서 나중에 보정해주기 
# 가족 여행 횟수 / 전체 여행 횟수 => 이 비율에 따라 가족 여행인지 아닌지에 따라 추가 점수 주기 
accompany_mapping = {
    '2인 여행(가족 외)' : 0,
    '자녀 동반 여행' : 1,
    '나홀로 여행' : 0,
    '3인 이상 여행(가족 외)' : 0,
    '2인 가족 여행' : 1,
    '부모 동반 여행' : 1,
    '3대 동반 여행(친척 포함)' : 1,
    '3인 이상 가족 여행(친척 포함)' : 1
}
lodging_data['TRAVEL_STATUS_ACCOMPANY'] = lodging_data['TRAVEL_STATUS_ACCOMPANY'].map(accompany_mapping)

# 대중교통 여행 횟수만 세서 나중에 보정해주기 
# 대중교통 여행 횟수 / 전체 여행 횟수 => 이 비율에 따라 대중교통 여행인지 아닌지에 따라 추가 점수 주기 
mvmn_map = {
    '자가용' : 0,
    '대중교통 등' : 1
}
lodging_data['MVMN_NM'] = lodging_data['MVMN_NM'].map(mvmn_map)

from sklearn.preprocessing import MinMaxScaler

# DGSTFN 정규화
scaler = MinMaxScaler()
lodging_data['NORMALIZED_DGSTFN'] = lodging_data['DGSTFN'] #scaler.fit_transform(lodging_data[['DGSTFN']])

# 보정 함수 적용
# REVISIT_INTENTION based on RCMDTN_INTENTION
lodging_data['CORRECTED_REVISIT_INTENTION'] = lodging_data['REVISIT_INTENTION'] * (
    1 - 0.1 * (lodging_data['RCMDTN_INTENTION'] < 3).astype(float)
)

# RCMDTN_INTENTION based on REVISIT_INTENTION
lodging_data['CORRECTED_RCMDTN_INTENTION'] = lodging_data['RCMDTN_INTENTION'] * (
    1 - 0.1 * (lodging_data['REVISIT_INTENTION'] < 3).astype(float)
)

# 가중치 계산
lodging_data['FINAL_RECOMMENDATION_SCORE'] = (
    0.20 * lodging_data['REVISIT_YN'].replace({'Y': 1, 'N': 0}) +
    0.35 * lodging_data['NORMALIZED_DGSTFN'] +
    0.25 * lodging_data['CORRECTED_REVISIT_INTENTION'] +
    0.20 * lodging_data['CORRECTED_RCMDTN_INTENTION']
)

# 한 숙소를 여러번 들린 경우도 있어서 그룹화하여 처리
grouped_lodgings = lodging_data.groupby(['VISIT_AREA_NM']).agg({
    'FINAL_RECOMMENDATION_SCORE' : ['mean', 'max', 'min'],
    'MVMN_NM' : 'sum',
    'TRAVEL_STATUS_ACCOMPANY' : 'sum',
    'X_COORD' : 'first',
    'Y_COORD' : 'first',
    'TRAVEL_ID' : 'count',
    'SLEEP' : 'sum'
}).reset_index()

grouped_lodgings.columns = ['VISIT_AREA_NM', 'AVG_SCORE', 'MAX_SCORE', 'MIN_SCORE', 'SUM_MVMN_TYPE', 'SUM_FAMILY_TPYE', 'X_COORD', 'Y_COORD', 'TOTAL_COUNT', 'TOTAL_SLEEP']

grouped_lodgings = grouped_lodgings[~grouped_lodgings['Y_COORD'].isna()]
grouped_lodgings

C:\Users\PC\AppData\Local\Temp\ipykernel_36020\2056487542.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lodging_data['VISIT_START_YMD'] = pd.to_datetime(lodging_data['VISIT_START_YMD'])
C:\Users\PC\AppData\Local\Temp\ipykernel_36020\2056487542.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lodging_data['VISIT_END_YMD'] = pd.to_datetime(lodging_data['VISIT_END_YMD'])
C:\Users\PC\AppData\Local\Temp\ipykernel_36020\2056487542.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,VISIT_AREA_NM,AVG_SCORE,MAX_SCORE,MIN_SCORE,SUM_MVMN_TYPE,SUM_FAMILY_TPYE,X_COORD,Y_COORD,TOTAL_COUNT,TOTAL_SLEEP
0,1004민박,2.400000,2.40,2.40,1,0,126.204859,34.946542,1,1
1,103랩,4.133333,4.20,4.00,3,0,129.117453,37.553213,3,1
2,141미니 호텔,3.300000,3.40,3.20,2,2,129.216402,35.842097,2,1
3,145게스트하우스,2.090000,2.09,2.09,2,0,126.915684,35.151952,2,1
4,1950하우스,3.550000,3.55,3.55,0,0,126.771044,33.501715,1,1
...,...,...,...,...,...,...,...,...,...,...
4486,힐스테이트에코 마곡나루역 오피스텔,3.800000,3.80,3.80,0,0,126.826784,37.568120,1,1
4487,힐튼 가든인 서울 강남점,4.000000,4.00,4.00,3,0,127.032788,37.485933,3,2
4488,힐튼 가든인 서울 강남점 조식 뷔페,2.900000,2.90,2.90,1,0,127.032788,37.485933,1,0
4489,힐튼 모텔,3.200000,3.20,3.20,1,0,126.384279,34.782018,1,1


In [5]:
import math

# X = 경도 / y = 위도
def haversine(lat1, lon1, lat2, lon2): 
    # 지구의 반지름 (킬로미터 단위)
    R = 6371.0
    
    # 위도와 경도를 라디안 단위로 변환
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # 위도와 경도의 차이 계산
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # 하버사인 공식 적용
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # 거리 계산
    distance = R * c
    
    return distance

def get_lodging_score_result(df, x_coord, y_coord, distance_threshold, mvmn, family):
    #거리 측정
    df['DISTANCE'] = df.apply(
        lambda row: haversine(y_coord, x_coord, row['Y_COORD'], row['X_COORD']),
        axis=1
    )
    
    filtered_df = df[df['DISTANCE'] <= distance_threshold]

    #점수 가산 
    filtered_df['FINAL_SCORE'] = filtered_df.apply(
        lambda row : row['AVG_SCORE'] 
        + ((row['SUM_MVMN_TYPE'] / row['TOTAL_COUNT']) if mvmn == 1 else ((row['TOTAL_COUNT'] - row['SUM_MVMN_TYPE']) / row['TOTAL_COUNT'])) * 0.5
        + ((row['SUM_FAMILY_TPYE'] / row['TOTAL_COUNT']) if family == 1 else ((row['TOTAL_COUNT'] - row['SUM_FAMILY_TPYE']) / row['TOTAL_COUNT'])) * 0.5
        , axis=1
    )
    return filtered_df

In [11]:
test_case = get_lodging_score_result(grouped_lodgings, 126.915684, 33.501715, 3, 1, 0)
test_case.sort_values(['FINAL_SCORE'], ascending=False).head(3)

C:\Users\PC\AppData\Local\Temp\ipykernel_36020\4143862360.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['FINAL_SCORE'] = filtered_df.apply(


,VISIT_AREA_NM,AVG_SCORE,MAX_SCORE,MIN_SCORE,SUM_MVMN_TYPE,SUM_FAMILY_TPYE,X_COORD,Y_COORD,TOTAL_COUNT,TOTAL_SLEEP,DISTANCE,FINAL_SCORE
3344,지미안 펜션,4.142857,4.2,4.0,7,0,126.906857,33.496810,7,3,0.983532,5.142857
698,다랑들락,4.100000,4.2,4.0,2,0,126.900933,33.495430,2,1,1.535947,5.100000
155,각자의 섬,4.000000,4.0,4.0,1,0,126.900496,33.490304,1,1,1.895635,5.000000


In [154]:
grouped_lodgings['TOTAL_COUNT'].sum()

12229

In [152]:
grouped_lodgings.sort_values('AVG_SCORE', ascending=False)

,VISIT_AREA_NM,AVG_SCORE,MAX_SCORE,MIN_SCORE,X_COORD,Y_COORD,TOTAL_COUNT,TOTAL_SLEEP
783,대운산 자연휴양림,2.800,2.800,2.800,129.208900,35.417856,2,1
3203,제주 마중,2.800,2.800,2.800,126.230303,33.389289,1,0
1752,비둘기 날 캠핑장,2.800,2.800,2.800,127.222078,38.077234,1,1
1313,매봉 힐링 쉼터 오토캠핑장,2.800,2.800,2.800,128.138758,37.260587,2,2
1753,비둘기날 캠핑장,2.800,2.800,2.800,127.222078,38.077234,1,1
...,...,...,...,...,...,...,...,...
2262,스파 리조트 안단테,0.405,0.405,0.405,127.042780,34.638173,2,1
983,동운장,0.405,0.405,0.405,127.727160,34.742757,2,1
3368,참 좋은 궁,0.405,0.405,0.405,127.617610,35.282205,1,1
2681,여수 하늘 보라 펜션,0.405,0.405,0.405,127.782944,34.669432,2,1
